In [ ]:
# mostly testing here, one semi-final version later on

In [ ]:
# use mongodb (actually gonna use pandas as it would be easier for others to use)

In [1]:
import os, datetime, json
import pandas as pd

# Testing

In [2]:
with open(os.path.join('C:\\Users','Joseph','AppData','LocalLow','Mediatonic','FallGuys_client','Player.log')) as f:
    lines = f.read()

In [3]:
lines = lines.split('\n')

episode_markers = []
for i, line in enumerate(lines):
    if '[CompletedEpisodeDto]' in line:
        episode_markers.append(i)

In [4]:
for j in episode_markers:
    final_lines = []
    temp_lines = lines[j:]
    for line in temp_lines:
        if '[ClientGameManager]' in line or ('>' not in line and '[' not in line): # end point
            break
        if line != "":
            final_lines.append(line)

In [5]:
final_lines

['04:45:47.638: == [CompletedEpisodeDto] ==',
 '> Kudos: 180',
 '> Fame: 81',
 '> Crowns: 0',
 '[Round 0 | round_gauntlet_04]',
 '> Qualified: True',
 '> Position: 25',
 '> Kudos: 30',
 '> Fame: 15',
 '> Bonus Tier: 2',
 '> Bonus Kudos: 35',
 '> Bonus Fame: 18',
 '> BadgeId: bronze',
 '[Round 1 | round_biggestfan]',
 '> Qualified: True',
 '> Position: 18',
 '> Kudos: 20',
 '> Fame: 10',
 '> Bonus Tier: 2',
 '> Bonus Kudos: 35',
 '> Bonus Fame: 18',
 '> BadgeId: bronze',
 '[Round 2 | round_tunnel]',
 '> Qualified: False',
 '> Position: 31',
 '> Kudos: 60',
 '> Fame: 20',
 '> Bonus Tier: ',
 '> Bonus Kudos: 0',
 '> Bonus Fame: 0',
 '> BadgeId: ']

In [12]:
def cleanLines(lines):
    return [line.replace('[', '').replace(']', '').replace('>', '').strip() for line in lines]

# splits show into show data and rounds
def roundSplit(lines):
    lines = cleanLines(lines)
    splits = []
    for i, line in enumerate(lines):
        if 'Round' in line:
            splits.append(i)
    
    splits.append(0)
    rounds = [lines[splits[i-1]:splits[i]] if i != len(splits)-1 else lines[splits[i-1]:]
              for i in range(1, len(splits))]
    
    return lines[1:splits[0]], rounds

showData, rounds = roundSplit(cleanLines(final_lines))
rounds

[['Round 0 | round_gauntlet_04',
  'Qualified: True',
  'Position: 25',
  'Kudos: 30',
  'Fame: 15',
  'Bonus Tier: 2',
  'Bonus Kudos: 35',
  'Bonus Fame: 18',
  'BadgeId: bronze'],
 ['Round 1 | round_biggestfan',
  'Qualified: True',
  'Position: 18',
  'Kudos: 20',
  'Fame: 10',
  'Bonus Tier: 2',
  'Bonus Kudos: 35',
  'Bonus Fame: 18',
  'BadgeId: bronze'],
 ['Round 2 | round_tunnel',
  'Qualified: False',
  'Position: 31',
  'Kudos: 60',
  'Fame: 20',
  'Bonus Tier:',
  'Bonus Kudos: 0',
  'Bonus Fame: 0',
  'BadgeId:']]

In [30]:
showData

['Kudos: 180', 'Fame: 81', 'Crowns: 0']

In [18]:
rounds_list = []
for round_ in rounds:
    round_dict = {'map': round_[0].split('|')[1], 'round_num':round_[0].split(' ')[1]}
    for line in round_[1:]:
        round_dict[line.split(':')[0]] = line.split(':')[1]
    rounds_list.append(round_dict)
rounds_list

[{'map': ' round_gauntlet_04',
  'round_num': '0',
  'Qualified': ' True',
  'Position': ' 25',
  'Kudos': ' 30',
  'Fame': ' 15',
  'Bonus Tier': ' 2',
  'Bonus Kudos': ' 35',
  'Bonus Fame': ' 18',
  'BadgeId': ' bronze'},
 {'map': ' round_biggestfan',
  'round_num': '1',
  'Qualified': ' True',
  'Position': ' 18',
  'Kudos': ' 20',
  'Fame': ' 10',
  'Bonus Tier': ' 2',
  'Bonus Kudos': ' 35',
  'Bonus Fame': ' 18',
  'BadgeId': ' bronze'},
 {'map': ' round_tunnel',
  'round_num': '2',
  'Qualified': ' False',
  'Position': ' 31',
  'Kudos': ' 60',
  'Fame': ' 20',
  'Bonus Tier': '',
  'Bonus Kudos': ' 0',
  'Bonus Fame': ' 0',
  'BadgeId': ''}]

# Final Part

In [184]:
# takes registered and connected lines
# returns start times for rounds
# (time at which game is found)
def getStartTimes(reg, conne):
    startTimes = []
    
    for i in range(len(conne)):
        registeredTime = datetime.datetime.strptime(reg[i].split('at: ')[-1], '%m/%d/%Y %H:%M:%S %p')
        connectedTime = datetime.datetime.strptime(conne[i].split(': [')[0], '%H:%M:%S.%f')
        
        d = (connectedTime - registeredTime)
        startTimes.append(registeredTime + (d - datetime.timedelta(days=d.days, hours=HOURS_DIFFERENTIAL)))
    
    return startTimes

In [189]:
# change these
list_of_finals = ['round_fallmountain', 'round_hex', 'round_royalfumble', 'round_jumpshowdown']
total_shows = 1 # read/write txt file
HOURS_DIFFERENTIAL = 6 # have to subtract 6

In [ ]:
with open(os.path.join('C:\\Users','Joseph','AppData','LocalLow','Mediatonic','FallGuys_client','Player.log')) as f:
    lines = f.read()

lines = lines.split('\n')

In [ ]:
with open(os.path.join('tash.txt')) as f:
    lines = f.read()

lines = lines.split('\n')

In [250]:
# get first line of each new show and usernames used for show
prevUser = '!!!!!!!!!!!!!!!'
lookUser = True
episode_markers = []
usernames = []
startRoundLines = []
userEndRoundLines = []
reg = []
conne = []
for i, line in enumerate(lines):
    if lookUser and 'Sending login request' in line:
        usernames.append(line.split(' player ')[-1].split(' networkID')[0].replace(',', ''))
        prevUser = usernames[-1]
        lookUser = False
    elif '[CompletedEpisodeDto]' in line:
        episode_markers.append(i)
        lookUser = True
    elif 'state from Countdown to Playing' in line:
        startRoundLines.append(line.split(': [')[0])
    elif '[ClientGameManager] Handling unspawn for player FallGuy' in line and prevUser in line:
        userEndRoundLines.append(line.split(': [')[0])
    elif '[QosManager] Registered' in line:
        reg.append(line)
    elif "[StateConnectToGame] We're connected to the server!" in line:
        conne.append(line)

# if no episodes found, end
if len(episode_markers) == 0:
    print('no episodes found')
        
# get time user spent in each round
# (time round starts until they either finish or are eliminated)
roundIdx = 0
roundTimes = []
for a, b in zip(startRoundLines, userEndRoundLines):
    roundTimes.append(str(datetime.datetime.strptime(b, '%H:%M:%S.%f') - datetime.datetime.strptime(a, '%H:%M:%S.%f')))

# gets start times for each show
startTimes = getStartTimes(reg, conne)

# **********************************************************
# for each show/episode ************************************
# **********************************************************
for showIdx, (j, user) in enumerate(zip(episode_markers, usernames)):
    this_show = total_shows
    total_shows += 1
    
    # get lines for this show
    final_lines = []
    temp_lines = lines[j:]
    for line in temp_lines:
        if line == '':
            continue
        if '[ClientGameManager]' in line or ('>' not in line and '[' not in line): # end point
            break
        if line != "":
            final_lines.append(line)
    
    # split data
    showData, rounds = roundSplit(final_lines)
    
    # set show data
    show_dict = {}
    show_dict['Show ID'] = this_show # id
    show_dict['Start Time'] = startTimes[showIdx]
    show_dict['Season'] = 2 # season, eventually offload to fcn 
    show_dict['Time Taken'] = getTimeTaken(startTimes[showIdx], final_lines[0].split(': ==')[0]) # approximate time taken
    show_dict['Final'] = rounds[-1][0].split('|')[-1].strip() in list_of_finals # final
    show_dict['Rounds'] = len(rounds) # num rounds
    show_dict['Username'] = user
    show_dict['addID'] = final_lines[0].split(': ==')[0] # end time
    # in group?
    
    # add other show data
    for line in showData:
        show_dict[line.split(':')[0]] = line.split(':')[1].strip()
    
    # ********************************************
    # get data for each round in show ************
    # ********************************************
    rounds_list = []
    
    # for each round in the show/episode
    for round_ in rounds:
        round_dict = {'Show ID': this_show, 
                      'Round Num':round_[0].split(' ')[1].strip(), 
                      'Map': round_[0].split('|')[1].strip()}
        round_dict['Time Spent'] = roundTimes[roundIdx]
        roundIdx += 1
        
        for line in round_[1:]:
            round_dict[line.split(':')[0]] = line.split(':')[1].strip()
        rounds_list.append(round_dict)
    
    # TODO save
    # save show_dict to one table
    # save each dict in rounds_list to another table
    
    

    
with open('totalshows.txt') as f:
    f.write(total_shows)

******************** 0 ********************
reg 2020-11-14 04:38:15
con 1900-01-01 04:39:30.981000
total 2020-11-13 22:39:30.981000


In [45]:
time_lines = []
for line in lines:
    if '[QosManager]' in line:
        print(line)
        time_lines
# Registered at seems to happen before match
# can match up first with first episode and go from there

# find match start and match end

04:37:50.885: [QosManager] Loaded 26 fallback results:
04:38:14.506: [QosManager] Periodic checks toggled - True
04:38:14.506: [QosManager] Updated next periodic check - 11/14/2020 4:43:14 AM
04:38:14.508: [QosManager] Attempting to trigger check
04:38:14.508: [QosManager] Triggering check
04:38:14.508: [QosManager] Starting server discovery
04:38:14.508: [QosManager] Updated next periodic check - 11/14/2020 4:43:14 AM
04:38:14.980: [QosManager] Starting to ping 26 servers
04:38:15.316: [QosManager] Pinging finished
04:38:15.316: [QosManager] Accepting results, no pre-existing accurate results exist
04:38:15.317: [QosManager] Increasing max valid accurate results received to 26
04:38:15.317: [QosManager] Saving 26 fallback results
04:38:15.317: [QosManager] Registered 26 results at: 11/14/2020 4:38:15 AM
04:38:15.317: [QosManager] Updated next periodic check - 11/14/2020 4:43:15 AM
04:39:00.368: [QosManager] Periodic checks toggled - False
04:39:30.820: [QosManager] Cancel ignored as _

In [50]:
with open(os.path.join('C:\\Users','Joseph','AppData','LocalLow','Mediatonic','FallGuys_client','Player.log')) as f:
    lines2 = f.read()

lines2 = lines2.split('\n')

In [186]:
for line in lines2:
    if 'Sending login request' in line:
        print(line)
    if '[GameSession]' in line:
        print(line)
    if '[CompletedEpisodeDto]' in line:
        print(line)
    if '[ClientGameManager] Handling unspawn for player FallGuy' in line and 'Fast Swimming Panther' in line:
        print(line)

07:50:11.458: [ClientGlobalGameState] Sending login request for player Fast Swimming Panther, networkID 3FB5D657
07:51:07.744: [GameSession] Changing state from Precountdown to Countdown
07:51:12.728: [GameSession] Changing state from Countdown to Playing
07:52:08.996: [ClientGameManager] Handling unspawn for player FallGuy [24] Fast Swimming Panther (FG.Common.MPGNetObject)
07:52:20.766: [GameSession] Changing state from Playing to GameOver
07:52:23.510: [GameSession] Changing state from GameOver to Results
07:52:41.378: [ClientGlobalGameState] Sending login request for player Fast Swimming Panther, networkID 3FB5D657
07:53:19.813: [GameSession] Changing state from Precountdown to Countdown
07:53:24.812: [GameSession] Changing state from Countdown to Playing
07:53:34.982: [ClientGameManager] Handling unspawn for player FallGuy [156] Fast Swimming Panther (FG.Common.MPGNetObject)
07:53:35.746: == [CompletedEpisodeDto] ==
07:53:53.378: [ClientGlobalGameState] Sending login request for p

In [104]:
episode_markers2 = []
usernames = []
prevUser = '!!!!!!!!!!!!!!!'
lookUser = True
startRoundLines = []
userEndRoundLines = []
for i, line in enumerate(lines2):
    if lookUser and 'Sending login request' in line:
        usernames.append(line.split(' player ')[-1].split(' networkID')[0].replace(',', ''))
        prevUser = usernames[-1]
        lookUser = False
    elif '[CompletedEpisodeDto]' in line:
        episode_markers2.append(i)
        lookUser = True
    elif 'state from Countdown to Playing' in line:
        startRoundLines.append(line.split(': [')[0])
    elif '[ClientGameManager] Handling unspawn for player FallGuy' in line and prevUser in line:
        userEndRoundLines.append(line.split(': [')[0])

# for each show/episode
for j, user in zip(episode_markers2, usernames):
    # get lines for this show
    final_lines2 = []
    temp_lines = lines2[j:]
    
    for line in temp_lines:
        if line == '':
            continue
        if '[ClientGameManager]' in line or ('>' not in line and '[' not in line): # end point
            break
        if line != "":
            final_lines2.append(line)
    
    # split data
    showData, rounds = roundSplit(final_lines2)
    print(showData)
print(usernames)

['> Kudos: 120', '> Fame: 60', '> Crowns: 0']
['> Kudos: 250', '> Fame: 115', '> Crowns: 0']
['Fast Swimming Panther', 'Fast Swimming Panther']


In [123]:
# get time
# usually an extra one (one more than number of shows played)
reg = []
conne = []
for line in lines2:
    if '[QosManager] Registered' in line:
        print(line)
        reg.append(line)
    if "[StateConnectToGame] We're connected to the server!" in line:
        print(line)
        conne.append(line)

07:49:15.442: [QosManager] Registered 26 results at: 11/14/2020 7:49:15 AM
07:50:11.457: [StateConnectToGame] We're connected to the server! Host = 107.181.173.201:8177
07:53:44.363: [QosManager] Registered 26 results at: 11/14/2020 7:53:44 AM
07:53:53.378: [StateConnectToGame] We're connected to the server! Host = 128.14.211.74:8077
08:02:02.220: [QosManager] Registered 26 results at: 11/14/2020 8:02:02 AM


In [180]:
startTimes = []
for i in range(len(conne)):
    print("*"*20, i, "*"*20)
    registeredTime = datetime.datetime.strptime(reg[i].split('at: ')[-1], '%m/%d/%Y %H:%M:%S %p')
    print('reg', registeredTime)
    connectedTime = datetime.datetime.strptime(conne[i].split(': [')[0], '%H:%M:%S.%f')
    print('con', connectedTime)
    d = (connectedTime - registeredTime)
    print('total', registeredTime + (d - datetime.timedelta(days=d.days, hours=HOURS_DIFFERENTIAL)))
    startTimes.append(registeredTime + (d - datetime.timedelta(days=d.days, hours=HOURS_DIFFERENTIAL)))

******************** 0 ********************
reg 2020-11-14 07:49:15
con 1900-01-01 07:50:11.457000
total 2020-11-14 01:50:11.457000
******************** 1 ********************
reg 2020-11-14 07:53:44
con 1900-01-01 07:53:53.378000
total 2020-11-14 01:53:53.378000


In [251]:
show_dict

{'Show ID': 11,
 'Start Time': datetime.datetime(2020, 11, 13, 22, 39, 30, 981000),
 'Season': 2,
 'Time Taken': '06:16.657000',
 'Final': False,
 'Rounds': 3,
 'Username': 'Fast Swimming Panther',
 'addID': '04:45:47.638',
 'Kudos': '180',
 'Fame': '81',
 'Crowns': '0'}

In [255]:
rounds_list[1]

{'Show ID': 11,
 'Round Num': '1',
 'Map': 'round_biggestfan',
 'Time Spent': '0:01:58.270000',
 'Qualified': 'True',
 'Position': '18',
 'Kudos': '20',
 'Fame': '10',
 'Bonus Tier': '2',
 'Bonus Kudos': '35',
 'Bonus Fame': '18',
 'BadgeId': 'bronze'}

### time per round

In [245]:
# time taken
d = datetime.datetime.strptime(final_lines[0].split(': ==')[0], '%H:%M:%S.%f') - startTimes[showIdx]
str(d - datetime.timedelta(days=d.days, hours=HOURS_DIFFERENTIAL))

'0:06:16.657000'

In [248]:
def getTimeTaken(start, end):
    d = datetime.datetime.strptime(end, '%H:%M:%S.%f') - start
    # start time was already adjusted for HOURS_DIFFERENTIAL, so need to again
    return str(d - datetime.timedelta(days=d.days, hours=HOURS_DIFFERENTIAL))[2:] # so hours isn't included

In [249]:
getTimeTaken(startTimes[showIdx], final_lines[0].split(': ==')[0])

'06:16.657000'

In [280]:
new_shows_df = pd.DataFrame(pd.Series(show_dict)).T
new_shows_df

,Show ID,Start Time,Season,Time Taken,Final,Rounds,Username,addID,Kudos,Fame,Crowns
0,11,2020-11-13 22:39:30.981,2,06:16.657000,False,3,Fast Swimming Panther,04:45:47.638,180,81,0


In [281]:
new_rounds_df = pd.DataFrame(rounds_list)
new_rounds_df

,Show ID,Round Num,Map,Time Spent,Qualified,Position,Kudos,Fame,Bonus Tier,Bonus Kudos,Bonus Fame,BadgeId
0,11,0,round_gauntlet_04,0:01:14.952000,True,25,30,15,2,35,18,bronze
1,11,1,round_biggestfan,0:01:58.270000,True,18,20,10,2,35,18,bronze
2,11,2,round_tunnel,0:00:10.102000,False,31,60,20,,0,0,


In [282]:
try:
    #load them
    shows_df = pd.read_csv(os.path.join('data', 'shows.csv'))
    rounds_df = pd.read_csv(os.path.join('data', 'rounds.csv'))
    
    # append
    shows_df = shows_df.append(new_shows_df, ignore_index=True)
    rounds_df = rounds_df.append(new_rounds_df, ignore_index=True)
except:
    # first time
    shows_df = new_shows_df
    rounds_df = new_rounds_df

# write them to their respective files
shows_df.to_csv(os.path.join('data', 'shows.csv'), index=False)
rounds_df.to_csv(os.path.join('data', 'rounds.csv'), index=False)